In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 59kB/s 
     |████████████████████████████████| 204kB 44.9MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=e9fe14992ced5edb10a06f192698cef6c1d0b90d1e797116a98741e14034ab05
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
from pyspark import SparkContext
import xgboost as xgb
sc=SparkContext.getOrCreate()
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split as ttsplit
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error as mse
import numpy as np 
import glob 
import csv


In [22]:
dataset=input('Wich Dataset(1,2,3 or 4): ').upper()
rep=input('Wich rep (F or P): ').upper()
if rep=='F': 
    rep='Frequency' 
elif rep=='P':
    rep='Presence'
else: 
    exit()
path='./drive/My Drive/sample_data/XGB/'+dataset+'/'+rep+'/*.csv'
files = glob.glob(path)
dic={}
for n in files:
	x=n.split('/')
	id=x[len(x)-1].split('.')
	dic[int(id[0])]= n
sorted_dict = {k: dic[k] for k in sorted(dic)}


Wich Dataset(1,2,3 or 4): 4
Wich rep (F or P): p


In [0]:
pd

<module 'pandas' from '/usr/local/lib/python3.6/dist-packages/pandas/__init__.py'>

In [6]:
print(len(sorted_dict))

16


In [23]:
list_accuracy=[]
params = {'objective': 'reg:squarederror', 'verbose': False}
x=0
for k,n in sorted_dict.items():
  p=n.replace('\\','/')
  win=pd.read_csv(p,header=None)
  win = win.sample(frac=1).reset_index(drop=True)
  num_col_chunk_train=len(win.columns)-1
  if x==0:
    print("initial model is created in chunk %d" %(x+1))
    #data_chunk=pd.DataFrame (dff[x])
    X_train_1= win.loc[:,0:(num_col_chunk_train-1)]
    y_train_1= win.loc[:,num_col_chunk_train]        
    xg_train_1 = xgb.DMatrix(X_train_1, label=y_train_1)
    model_1 = xgb.train(params, xg_train_1, 1)
    model_1.save_model('model_1.model')
  else:
    print("testing chunk %d" %(x+1))
    #data_chunk=pd.DataFrame (dff[x])
    X_train_1= win.loc[:,0:(num_col_chunk_train-1)]
    y_train_1= win.loc[:,num_col_chunk_train]
    X_test=X_train_1
    y_test=y_train_1
    xg_test = xgb.DMatrix(X_test, label=y_test)
    print("MSE for this chunk:",mse(model_1.predict(xg_test), y_test))     # benchmark  
        
    #prediction
    a=model_1.predict(xg_test).tolist()
    accuracy_matrix = np.around(a,decimals=0)-y_test
    accuracy= (len(accuracy_matrix)-np.sum(np.absolute(accuracy_matrix)))/len(accuracy_matrix)
    list_accuracy=np.append(list_accuracy,accuracy)
    print("Accuracy for this chunk: %s",accuracy)     # benchmark  
    print("Accummulated Accuracy for this chunk: %s",np.average(list_accuracy))     # benchmark  
    print("training chunk %d" %(x+1), "and model is updated")
        
    #training
    xg_train_1 = xgb.DMatrix(X_train_1, label=y_train_1)
    model_1 = xgb.train(params, xg_train_1, 1, xgb_model='model_1.model')
    model_1.save_model('model_1.model')
  x=x+1



initial model is created in chunk 1


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


testing chunk 2
MSE for this chunk: 1.054655542828573
Accuracy for this chunk: %s 0.06666666666666667
Accummulated Accuracy for this chunk: %s 0.06666666666666667
training chunk 2 and model is updated


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


testing chunk 3
MSE for this chunk: 0.5604061473180457
Accuracy for this chunk: %s 0.041666666666666664
Accummulated Accuracy for this chunk: %s 0.05416666666666667
training chunk 3 and model is updated


...

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


testing chunk 20
MSE for this chunk: 0.186206416466862
Accuracy for this chunk: %s 0.6857142857142857
Accummulated Accuracy for this chunk: %s 0.6153111852328844
training chunk 20 and model is updated


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


testing chunk 21
MSE for this chunk: 0.12334598768855151
Accuracy for this chunk: %s 0.8275862068965517
Accummulated Accuracy for this chunk: %s 0.6259249363160678
training chunk 21 and model is updated


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [0]:
win.shape
win.columns
len(win.columns)-1